# Metody Obliczeniowe w Nauce i Technice
# Laboratorium 2: Rozwiązywanie układów równań liniowych
## Przemysław Roman

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import time
from copy import deepcopy

def time_exec(func, *args):
    start_time = time.time()
    result = func(*args)
    return result, time.time() - start_time

## Zadanie 1 Metoda Gaussa-Jordana

Napisz i sprawdź funkcję rozwiązującą układ równań liniowych n × n metodą Gaussa-
Jordana. Dla rozmiarów macierzy współczynników większych niż 500 × 500 porównaj
czasy działania zaimplementowanej funkcji z czasami uzyskanymi dla wybranych funkcji
bibliotecznych.

Funkcje pomocnicze

In [61]:
# układa rzędy tak aby ilość wiodących zer w rzędach była niemalejąca idąc od góry do dołu układu równań
def order_rows_by_leading_zeros(AB, n):
  pre_zeros = [0] * n

  for i in range(n):
    for j in range(n):
      if AB[i,j] != 0:
        break
      pre_zeros[i] += 1

  sorted_indexes = [i for _, i in sorted(zip(pre_zeros, range(n)))]
  return np.array([AB[i] for i in sorted_indexes])

# sprawia, że wszystkie elementy w tablicy są w zakresie [0, 1]
def scaling(AB, n):
    for i in range(n):
        _max = max(AB[i])
        AB[i] /= np.max(AB[i])

def find_max_abs_scalar_i(AB, n, j):
    max_i = -1
    for i in range(j+1, n):
        if AB[i,j] > AB[max_i,j]:
            max_i = i
    return max_i

def partial_pivoting(AB, n, i):
    max_i = find_max_abs_scalar_i(AB, n, i)
    if max_i != -1:
        AB[[i, max_i]] = AB[[max_i, i]]

Implementacja algorytmu Gaussa-Jordana

In [62]:
# i - rząd, j - kolumna
def gauss_jordan(_AB, n):
    AB = deepcopy(_AB)
    scaling(AB, n)
    # AB = order_rows_by_leading_zeros(AB, n)

    for i in range(n):
        partial_pivoting(AB, n, i)

        if AB[i,i] != 0:
            # redukcja pivotu do 1
            scalar = 1 / AB[i,i]
            AB[i] *= scalar

            # redukcja współczynników do 0, które znajdują się w tej samej kolumnie co pivot
            for k in range(n):
                if k != i:
                    scalar = AB[k,i] / AB[i,i]
                    AB[k] -= scalar * AB[i]

    return AB[:,-1] # ostatnia kolumna zawiera wyniki

Dane testowe

In [63]:
np.random.seed(17)
n = 500
A = np.random.random((n, n))
B = np.random.random((n, 1))
AB = np.concatenate((A, B), axis=1)

Testy poprawności rozwiązania oraz czasu wykonania

In [64]:
def get_absolute_errors_sum(vals, expected_vals):
    return np.sum(np.absolute(vals - expected_vals))

X1, t1 = time_exec(gauss_jordan, AB, n)
print(f'Moja implementacja: {t1}[s]')
X2, t2 = time_exec(np.linalg.solve, A, B)
X2 = X2.flatten()
print(f'np.linalg.solve: {t2}[s]')
print('Sumaryczny błąd względny:', get_absolute_errors_sum(X1, X2))

Moja implementacja: 0.975275993347168[s]
np.linalg.solve: 0.002681732177734375[s]
Sumaryczny błąd względny: 3.808764805282583e-11


In [11]:
import numpy as np

def gauss_jordan_full_pivoting(W, N):
    A = W.copy()
    B = N.copy()
    n = len(A)
    permutations = []
    for i in range(n):
        # get indices of max element in sub-matrix
        m_row, m_col = np.unravel_index(A[i:, i:].argmax(), [n - i, n - i])
        # translate indices to initial matrix
        m_row  += i
        m_col  += i
        # print(A)
        # print(f"i = {i}")


        if i != m_row: # swap rows
            # print(f"swapping row: {m_row} <-> {i}")
            A[[i, m_row]] = A[[m_row, i]]
            B[i], B[m_row] = B[m_row], B[i]
            # print(A, "after swap")
            # print(B, "B after swap")

        if i != m_col: #swap columns
            # print(f"swapping col: {m_col} <-> {i}")
            A[:, [i, m_col]] = A[:, [m_col, i]]
            permutations.append((i, m_col))
            # B[i], B[m_col] = B[m_col], B[i]
            # print(A, "after swap")

        scale = A[i, i]
        A[i] /= scale
        B[i] /= scale

        for j in range(n):
            if i != j:
                B[j] -= B[i] * A[j, i]
                A[j] -= A[j, i] * A[i]
                A[j, i] = 0

        # print(A, "A")
        # print(B, "B")

    for i,j in permutations:
        B[i], B[j] = B[j], B[i]

    # print(A)
    return B

In [12]:
X, t = time_exec(gauss_jordan_full_pivoting, A, B)
X, t

(array([[-6.51025623e+01],
        [ 3.76951290e+01],
        [-2.29312366e+02],
        [-3.34813737e+02],
        [ 1.33809363e+02],
        [-6.41013037e+00],
        [ 1.13332097e+02],
        [ 2.93565288e+01],
        [ 7.16912581e+01],
        [ 1.76460154e+02],
        [-1.15453323e+02],
        [-1.96902490e+01],
        [-8.66443447e+01],
        [-2.75596349e+00],
        [ 2.85540116e+01],
        [-4.65731620e+01],
        [ 3.10550467e+02],
        [-2.73232512e+02],
        [-1.06369350e+02],
        [-2.66626708e+02],
        [-9.01502649e+01],
        [ 3.76951290e+01],
        [-1.61519810e+02],
        [ 3.58519823e+02],
        [-4.29693699e+01],
        [-4.65147294e+01],
        [-4.00664043e+01],
        [-1.80368009e+02],
        [ 1.91802392e+01],
        [ 6.96058389e+01],
        [ 3.86696221e+01],
        [ 7.44828833e+01],
        [-1.81377949e+02],
        [-1.39016698e+01],
        [-2.64044916e+02],
        [ 2.81357634e+00],
        [-1.41818545e+02],
 

In [9]:
gauss_jordan(AB, n)

array([ 8.37342460e-01,  1.07245357e-01, -3.39958435e-01, -1.33310637e+00,
        1.46873061e+00, -7.55373430e-01, -1.34093601e+00,  5.89612915e-02,
       -9.84589706e-01, -3.12915747e-01, -1.82826814e+00, -1.70235263e-01,
        6.58210778e-02, -1.81197688e+00,  7.97799527e-01,  9.39612105e-01,
       -7.47207407e-02,  4.25482581e-02, -8.82490963e-01, -5.83423670e-01,
       -2.74541666e-01, -6.63687518e-01,  1.25127235e+00, -7.09848311e-01,
       -2.61378039e+00, -1.39445905e+00,  8.72941046e-01, -2.34028705e+00,
        3.83305473e-01,  1.22232925e+00,  7.68375661e-01, -4.18975697e-02,
       -4.82853808e-01, -1.44185057e+00,  1.12499494e+00, -1.11837468e+00,
       -9.00274507e-02,  1.65245484e+00,  1.61447844e+00,  6.04379524e-01,
       -5.58375098e-01,  1.18696618e-01,  1.68602847e+00,  3.23492414e-01,
        6.57306607e-01, -6.20227454e-01,  1.57644770e-01,  6.62609106e-01,
       -6.54231904e-01, -1.26011574e-01,  6.51108279e-01, -1.03564910e-01,
        6.84298804e-01,  